# Path Following Example

Using the path computed in MapAnnotation notebook, the movement speed computed in MotionDynamics notebook, the localization in the Localization notebook and the angle keeping in the FeedbackLoop notebook.

In [30]:
import cv2 as cv
from PIL import ImageGrab
from dataclasses import dataclass
import numpy as np
from matplotlib import pyplot as plt
import math
import pyautogui
from pynput import keyboard, mouse
import threading
import time

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from model.angle import compute_angle
from PIL import Image, ImageTk, ImageDraw, ImageFont, ImageGrab

import pyautogui
from dataclasses import dataclass
import time
import random

In [67]:
@dataclass
class Instruction:
    angle: int
    dist: int

In [20]:
# 720p
ARROW_TOP_LEFT_X = 105
ARROW_TOP_LEFT_Y = 185
ARROW_BOTTOM_RIGHT_X = 130
ARROW_BOTTOM_RIGHT_Y = 210
ARROW_BBOX = (ARROW_TOP_LEFT_X, ARROW_TOP_LEFT_Y, ARROW_BOTTOM_RIGHT_X, ARROW_BOTTOM_RIGHT_Y)
screen = ImageGrab.grab(bbox=ARROW_BBOX)
angle = compute_angle(screen)
print(angle)
screen

-1.1661643524420398


In [21]:
import pyautogui
from dataclasses import dataclass
import time
import random

@dataclass
class Coordinate:
    x: int
    y: int

    def __add__(self, other):
        return Coordinate(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Coordinate(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar):
        return Coordinate(self.x * scalar, self.y * scalar)

styles = [
    pyautogui.easeInQuad,     # start slow, end fast
    pyautogui.easeOutQuad,    # start fast, end slow
    pyautogui.easeInOutQuad,  # start and end fast, slow in middle
    pyautogui.easeInBounce,   # bounce at the end
    pyautogui.easeInElastic  # rubber band at the end
]

def drag(start: Coordinate, end: Coordinate, duration_ms: int):
    pyautogui.moveTo(start.x, start.y)
    pyautogui.dragTo(end.x, end.y, duration_ms/1000, random.choice(styles), button='left')

def drag_no_noise(start: Coordinate, end: Coordinate, duration_ms: int):
    pyautogui.moveTo(start.x, start.y)
    pyautogui.dragTo(end.x, end.y, duration_ms/1000, button='left')

def activate():
    pyautogui.click(600, 450)

In [57]:
# small movements
CENTER = Coordinate(642, 200)
DELTA = Coordinate(30, 0)
LEFT = CENTER - DELTA
RIGHT = CENTER + DELTA

# big movements
BIG_LEFT = CENTER - DELTA*3
BIG_RIGHT = CENTER + DELTA*3

IGNORE_THRESHOLD = 5
BIG_THRESHOLD = 20

def set_angle(current, target):
    activate()
    current += 360
    target += 360

    gap = abs(target - current)
    if target > current:
        # look left
        approx_rotate_left(target - current)
        return
    else:
        # look right
        approx_rotate_right(current-target)

# Maintain within 5 degrees
def keep_angle(current, target):
    activate()
    current += 360
    target += 360

    gap = abs(target - current)
    
    if gap < IGNORE_THRESHOLD:
        return

    left = LEFT
    right = RIGHT
    if gap > BIG_THRESHOLD:
        # ignore
        return
        
        # if target > current:
        #     # look left
        #     approx_rotate_left(target - current)
        #     return
        # else:
        #     # look right
        #     approx_rotate_right(current-target)
        
    if target > current:
        # need to look left
        drag(CENTER, left, 300)
    else:
        # need to look right
        drag(CENTER, right, 300)

In [62]:
styles = [
    pyautogui.easeInQuad,     # start slow, end fast
    pyautogui.easeOutQuad,    # start fast, end slow
    pyautogui.easeInOutQuad,  # start and end fast, slow in middle
    # pyautogui.easeInBounce,   # bounce at the end
    # pyautogui.easeInElastic  # rubber band at the end
]

def drag(start: Coordinate, end: Coordinate, duration_ms: int):
    pyautogui.moveTo(start.x, start.y)
    pyautogui.dragTo(end.x, end.y, duration_ms/1000, random.choice(styles), button='left')

In [63]:
CENTER = Coordinate(642, 200)
DELTA_X = 257

def approx_rotate_right(degrees):
    scaling = degrees/90

    RIGHT = Coordinate(CENTER.x + (DELTA_X * scaling), CENTER.y)
    activate()
    drag(CENTER, RIGHT, 500)

def approx_rotate_left(degrees):
    scaling = degrees/90

    LEFT = Coordinate(CENTER.x - (DELTA_X * scaling), CENTER.y)
    activate()
    drag(CENTER, LEFT, 500)

In [69]:
def activate():
    pyautogui.click(600, 450)

def holdKey(key, duration):
    # todo: adjust based on global coordinate prediction
    pyautogui.keyDown(key)
    
    time.sleep(duration)
    pyautogui.keyUp(key)

def holdAngle(target_angle, duration = None):
    # todo: ignore bad angle frames once aligned for multiple frames
    start = time.time()
    while True:
        end = time.time()
        if duration is not None and (end - start > duration):
            break
        
        arrow = ImageGrab.grab(bbox=ARROW_BBOX)
        angle = compute_angle(arrow)
        if angle:
            keep_angle(angle, target_angle)

        if duration is None:
            return

def initAngle(target_angle):
    # face FOV
    activate()
    pyautogui.keyDown('w')
    time.sleep(0.05)
    pyautogui.keyUp('w')

    # one-time angle correction
    arrow = ImageGrab.grab(bbox=ARROW_BBOX)
    angle = compute_angle(arrow)
    if angle:
        set_angle(angle, target_angle)

    pyautogui.keyDown('w')
    time.sleep(0.05)
    pyautogui.keyUp('w')

PIXELS_PER_SECOND = 15


instructions = [
    Instruction(0, 181),
    Instruction(90, 83)
]

for inst in instructions:
    target_angle = inst.angle
    duration = inst.dist / PIXELS_PER_SECOND

    initAngle(target_angle)
    
    KeyboardThread = threading.Thread(target=holdKey, args=('w', duration,))
    MouseThread = threading.Thread(target=holdAngle, args=(target_angle, duration,))

    activate()
    KeyboardThread.start()
    MouseThread.start()
    
    KeyboardThread.join()
    MouseThread.join()